In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:

## Libraries Required

 # Huggingface hub has various repositories by which we call various opensource llm models
!pip install langchain-huggingface 
## For API Calls
!pip install huggingface_hub  

# if if go head and search for any of llm model one of the feature of calling this particular model by is using transformers pipeline
!pip install transformers    
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain

In [2]:
# from langchain. import ChatOpenAI
from langchain import PromptTemplate, HuggingFaceHub, LLMChain  # ascessing the llms models of openAI API by langchain framework
from langchain_huggingface import HuggingFaceEndpoint

## Different Types of collecting OpenAI Key

### Using the OS Module to Access Environment Variables
### Avoid Direct Pasting
- Use os module to read environment variables securely.
### Method to Access Environment Variable
- Use os.getenv to read the variable.


- import os
- openai_key = os.getenv('OPENAI_API_KEY')


### Creating Environment Variables
#### Windows Environment Variable:

- Search "Environment Variables" in Windows.
- Add the key-value pair.

#### Temporary Export in Terminal:

- Use export command:

- export OPENAI_API_KEY='your_openai_key'

### Creating a .env File
- Create .env file in project directory:


- OPENAI_API_KEY="your_openai_key"


### Reading from .env File
Use python-dotenv to read .env file:


from dotenv import load_dotenv
import os

- load_dotenv()
- openai_key = os.getenv('OPENAI_API_KEY')

In [3]:
from dotenv import load_dotenv  # Python-dotenv reads key-value pairs from a .env file and can set them as environment variables., we use this library to read the open AI api key secretly rather than exposing it to public it helps us to read the key in secrete and than after mentioning the key value pair of openai key in .env file makesure add this .env file to .gitiognore so it wont get push to github

load_dotenv()  # take environment variables from .env.

True

In [4]:
# reading the hugginface api key token from .env file 
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")


### Importing the LLM model  from hugging face

In [5]:
# mistralai/Mistral-7B-Instruct-v0.2 , microsoft/Phi-3-mini-4k-instruct
# now i have created object for the HuggingfaceHUb class in that object i can able to collect the specified repo_id of model  in llm variable
llm =HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature= 0.5,
    token=KEY
)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
c:\Users\mahen\anaconda_new\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\mahen\.cache\huggingface\token
Login successful


In [ ]:
print(llm) # as we can see we have imported llm model from hugging face 

In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

### Working with LLMs Using LLMChain Framework

1. **Introduction to LLM Usage**
   - Accessing LLM (GPT model) via Hugging face API using LLMChain framework.
   - Importance of designing input and output prompts for interaction with the model.

2. **Designing Input Prompts**
   - To interact with the LLM, an input prompt is necessary.
   - The input prompt is crafted to guide the model on what to generate.

3. **Output Response from LLM**
   - The output from the LLM is also considered a prompt.
   - The design of input and output prompts is crucial for obtaining the desired results.

4. **Project Context: MCQ Generator**
   - The goal is to generate Multiple Choice Questions (MCQs) using GPT models.
   - Input: A paragraph or text related to a subject (e.g., data science).
   - Output: MCQs generated based on the input text.

5. **Example Scenario**
   - Providing a paragraph related to data science to the GPT model.
   - The model processes the paragraph and generates relevant MCQs.

6. **Steps to Design Prompts**
   - Use a prompt template to design input prompts.
   - The output from the model will be a well-structured prompt containing the MCQs.

### creating Chain 1 which includes the prompt template1 + llm and prompt template dependends on Template and input varible which user will pass

In [8]:
# formatting the question of input prompt in desired formate we are making our model to learn this customized question for generating the mcqs output by the llm model
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Ensure to make {number} MCQs 
"""

In [9]:
# creating a PromptTemplate 1
 
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"], # these input variables is going to enter by users for generting mcqs of document by the llm model
    template=TEMPLATE
    )

In [10]:
# creating the  chain 1 = we use llm chain for connecting serval components here in our case we are going to connect llm +prompt template1
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\mahen\anaconda_new\envs\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### creating Chain 2 which includes the prompt template 2 + llm and prompt template 2 dependends on Template and input varible which user will pass

In [11]:
# i have designed another template formate which this pattern question analyze and evaluate the output response answers of template 1 means this template 2 analyze whether the answers of the mcqs of the template 1 of response json output given correct or not my template 2 will evaluate once the mcqs got generated by template 1
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz if the students will be able to understand the questions and answer them. Only use at max 50 words for complexity analysis.
And make sure dont generate this thing Best regards, [Your Name] and also keep in mind that generate the mcqs in good formate and give a space between them while generating output 
Check from an expert English Writer of the below quiz:
{quiz}
"""

In [12]:
# here i have created the prompt template 2 for the template2 which it takes the input variables from the user and those input varibles get passed to the template and our llm model based on the template formate it generates output 
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)
print(quiz_evaluation_prompt)

input_variables=['quiz', 'subject'] template='\nYou are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\nYou need to evaluate the complexity of the question and give a complete analysis of the quiz if the students will be able to understand the questions and answer them. Only use at max 50 words for complexity analysis.\nAnd make sure dont generate this thing Best regards, [Your Name] and also keep in mind that generate the mcqs in good formate and give a space between them while generating output \nCheck from an expert English Writer of the below quiz:\n{quiz}\n'


### Creating chain 2 with prompt Template 2 + LLMm

In [13]:
# Now creating chain2 which it connects  the quiz prompt evaluation + llm 
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True) # here i connceted the llm with prompt which evaluates the mcqs based on the output key variable 

###  Connecting chain 1 + chain 2 with Sequential chain

In [14]:
# By using SequentiaChain concept iam going to connect the 2 chains
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone"],
                                        output_variables=["quiz", "review"], verbose=True,) # here in the chain argument we passed the quiz_chain,review_chain so the quiz_chain generates the mcqs by the llm which it uses prompt template 1and review chain evaluates the generated mcqs by llm which it uses the prompt template 2 

### Taking the input data content inorder to read the input data content by our llm model

In [15]:
file_path=r"C:\datascience End to End Projects\generative-MCQ-generator-with-langchain-OpenAI-LLM\data.txt"

In [16]:
file_path

'C:\\datascience End to End Projects\\generative-MCQ-generator-with-langchain-OpenAI-LLM\\data.txt'

In [17]:
## reading the file input data content 
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebb’s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied on human cognitive 

In [19]:
# here this is the input paramters we need to pass to the sequential chain in order to generate the outputs of 2 chains , iam consider as user and i have given the parameters for generating the mcqs 
NUMBER=5 
SUBJECT="machine learning"
TONE="simple" # tone stands for temperature 


In [20]:
# Passing the argumnets to the generate_evaluate_chain which will furthur pass to llm model to generate the output
response=generate_evaluate_chain(
            {
                "text": TEXT,
                "number": NUMBER,
                "subject":SUBJECT,
                "tone": TONE
            }
            ) 
 

c:\Users\mahen\anaconda_new\envs\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebb’s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artifici

In [21]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\\n\\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebb’s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied on huma

In [24]:
quiz = response.get("quiz")
print(quiz)

Here is the quiz:

1. Who coined the term "Machine Learning" in 1959?
A) Arthur Samuel
B) Donald Hebb
C) Walter Pitts
D) Warren McCulloch

Answer: A) Arthur Samuel

2. What was the name of the book published by Donald Hebb in 1949?
A) The Organization of Behavior
B) Learning Machines
C) Cybertron
D) The Art of Machine Learning

Answer: A) The Organization of Behavior

3. What was the name of the experimental "learning machine" developed by Raytheon Company in the early 1960s?
A) Cybertron
B) Learning Machine
C) Pattern Recognition
D) Reinforcement Learning

Answer: A) Cybertron

4. Who provided a widely quoted definition of the algorithms studied in the machine learning field?
A) Tom M. Mitchell
B) Alan Turing
C) Donald Hebb
D) Arthur Samuel

Answer: A) Tom M. Mitchell

5. What are the two primary objectives of modern-day machine learning?
A) To classify data and make predictions
B) To make predictions and classify data
C) To analyze data and make decisions
D) To learn from experience 